Following https://github.com/bigcode-project/starcoder

In [1]:
# Step 1: Clone the repo and setup environment

import os
import subprocess

repo_dir = "./tamarind-finetune"
repo_url = "https://github.com/smartrics/tamarind-finetune.git"

if os.path.isdir(repo_dir):
    print("Directory 'tamarind-finetune' exists. Pulling latest changes...")
    subprocess.run(["git", "-C", repo_dir, "pull"], check=True)
else:
    print("Directory 'tamarind-finetune' does not exist. Cloning repository...")
    subprocess.run(["git", "clone", repo_url, repo_dir], check=True)
print("finished!")

Directory 'tamarind-finetune' does not exist. Cloning repository...
finished!


In [2]:
%cd ./tamarind-finetune

/content/tamarind-finetune


We’ll finetune `bigcode/starcoderbase-1b`, which is a 1B parameter model trained on 80+ programming languages. This is a gated model, so if you plan to run this notebook with this exact model, you’ll need to gain access to it on the model’s page. Log in to your Hugging Face account to do so:

In [ ]:
!conda install pytorch torchvision torchaudio pytorch-cuda -c pytorch -c nvidia
!conda install -c huggingface transformers
!pip install git+https://github.com/huggingface/peft.git
!conda install -c huggingface -c conda-forge datasets
!conda install -c conda-forge accelerate
!conda install -c conda-forge huggingface_hub
!pip install bitsandbytes
!pip install wandb

In [3]:
from huggingface_hub import notebook_login

# --- 2. Login to Hugging Face Hub ---
notebook_login()

In [ ]:
import wandb
wandb.login()

In [ ]:
!python data_starcoderbase/finetune.py \
  --model_path="bigcode/starcoderbase-1b"\
  --dataset_path="./data_starcoderbase/tamarind_data.csv" \
  --subset="data/finetune"\
  --split="train"\
  --size_valid_set 10000\
  --seq_length 8192\
  --max_steps 1000\
  --batch_size 1\
  --input_column_name="question"\
  --output_column_name="response"\
  --gradient_accumulation_steps 16\
  --learning_rate 1e-4\
  --lr_scheduler_type="cosine"\
  --num_warmup_steps 100\
  --weight_decay 0.05\
  --output_dir="./checkpoints"

The size of the SE dataset is better manageable when using streaming. We also have to precise the split of the dataset that is used. For more details, check the dataset's page on 🤗. Similarly we can modify the command to account for the availability of GPUs

In [ ]:
!python data_starcoderbase/merge_peft_adapters.py \
   --base_model_name_or_path "bigcode/starcoderbase-1b"
   --peft_model_path "checkpoints/checkpoint-1000"
   --peft_model_path "smartrics/starcoderbase-1b-tamarind"
   --push_to_hub

